In [2]:
import ee
ee.Initialize()


In [4]:
import ee
ee.Initialize()
def get_utm_zone_and_crs( lat, lon):
    """
    Calculate the UTM zone number from latitude and longitude.

    Parameters:
        lat (float): Latitude in decimal degrees.
        lon (float): Longitude in decimal degrees.

    Returns:
        int: UTM zone number (1 to 60).
    """
    if lat < -80 or lat > 84:
        raise ValueError("Latitude must be between -80 and 84 degrees.")

    # Calculate the UTM zone number
    zone_number = int((lon + 180) / 6) + 1
    # Determine hemisphere and corresponding CRS code
    if lat >= 0:
        hemisphere = "Northern"
        crs_code = f"EPSG:326{zone_number:02d}"  # Northern Hemisphere
    else:
        hemisphere = "Southern"
        crs_code = f"EPSG:327{zone_number:02d}"  # Southern Hemisphere
    
    return crs_code
workspace = r'G:\temp'
extent_gcs = '-92.0354,32.6134,-89.9583,34.1614'
s1_date = '2018-02-10,2018-02-19'
url_jrc="JRC/GSW1_4/GlobalSurfaceWater"
jrc_band = "occurrence"
s1_band = "VV"
url_s1="COPERNICUS/S1_GRD"
download_s1=True
download_jrc=True
numbers = extent_gcs.split(',')
s1_date = s1_date.split(',')

# Assign the extracted numbers to variables
x1, y1, x2, y2 = map(float, numbers)
bbox = ee.Geometry.BBox(x1,y1,x2,y2)
# Example coordinates (for the given image center)
latitude = y1
longitude = x1

# Get UTM zone and CRS code
crs_code = get_utm_zone_and_crs(latitude, longitude)
print(f'Using computed UTM zone {crs_code} for the image spaptial reference to avoid distortions')
crs = crs_code
# send the task for jrc on gee for processing. Establish a watching thread to update the status until the task is completed
if(url_jrc != ""):
    try:
        datasetJRC = ee.Image(url_jrc).select(jrc_band).clip(bbox)
        if download_jrc:
            task = ee.batch.Export.image.toDrive(
                image= datasetJRC,
                description= 'JCRwaterpixels',
                region= bbox,
                folder= 'AnyFolder',
                scale= 10,
                fileFormat= 'GeoTIFF',
                maxPixels= 3784216672400,
                crs= crs
                )
            task.start()
            print(f"Export task started with ID: {task.id}")
            print(f"Export destination: JCRwaterpixels.tif in AnyFolder")

    except Exception as e:
        print(e)

Using computed UTM zone EPSG:32615 for the image spaptial reference to avoid distortions
Export task started with ID: 2YLDBF3FRANGR5ZFJCGGEVTF
Export destination: JCRwaterpixels.tif in AnyFolder


In [17]:
from osgeo import gdal

# Path to the source raster (change to your source file or in-memory dataset)
source_file = r"G:\temp\jrc_watermask.tif"  # replace with your source file

# Define the output projection (for example, UTM zone 33N, EPSG:32633)
target_crs = 'EPSG:32633'

# Warp the source file to an in-memory raster with the specified target projection
in_memory_raster = gdal.Warp('/vsimem/temp_mosaic', source_file, dstSRS=target_crs)

In [90]:
import ee
ee.Initialize()
limit = [34.157424235794025, -84.07053260218785, 34.247255764205974, -83.96191679781214]
region = ee.Geometry.BBox(limit[1],limit[0],limit[3],limit[2])
image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(region)
              .filterDate('2020-04-01', '2020-09-01')
              .median())
proj = image.projection().getInfo()
if 'crs' in proj:
    proj_crs = proj['crs']
elif 'wkt' in proj:
    proj_crs = proj['wkt']
p = ee.Projection(proj_crs, proj['transform'])
scale = p.nominalScale().getInfo()
band = image.getInfo()['bands'][0]
crs = band['crs']
crs_transform = band['crs_transform']
transform = band['crs_transform']
print(transform)

[1, 0, 0, 0, 1, 0]


In [91]:
from pyproj import Proj
import numpy as np
transform = [1, 0, 0, 0, 1, 0]
limit = [34.157424235794025, -84.07053260218785, 34.247255764205974, -83.96191679781214]
proj_crs = "EPSG:4326"
wp = Proj(proj_crs)
bbox = np.asarray(wp((limit[1], limit[3], limit[3], limit[1]),
                (limit[0], limit[0], limit[2], limit[2]), ))
## move to nearest pixel - 60 m works for both S2 and Landsat
bbox -= bbox % 60
## ROI x and y ranges
xrange = np.asarray((min(bbox[0]), max(bbox[0])))
yrange = np.asarray((max(bbox[1]), min(bbox[1])))
xdim = (xrange[1]-xrange[0])/transform[0]
ydim = (yrange[1]-yrange[0])/transform[4]
## ROI x and y pixel ranges
xprange = (xrange-transform[2]) / transform[0]
yprange = (yrange-transform[5]) / transform[4]
rdim = int(xprange[1]-xprange[0]), int(yprange[1]-yprange[0])
## origin of subset in image
porigin = xprange[0], yprange[0]

In [93]:
wp

<Other Coordinate Operation Transformer: longlat>
Description: PROJ-based coordinate operation
Area of Use:
- undefined

In [89]:
image.getInfo()

{'type': 'Image', 'bands': []}

In [99]:
def get_utm_zone_and_crs(lat, lon):
    # Calculate UTM zone number
    zone_number = int((lon + 180) / 6) + 1
    
    # Determine hemisphere and corresponding CRS code
    if lat >= 0:
        hemisphere = "Northern"
        crs_code = f"EPSG:326{zone_number:02d}"  # Northern Hemisphere
    else:
        hemisphere = "Southern"
        crs_code = f"EPSG:327{zone_number:02d}"  # Southern Hemisphere
    
    return zone_number, hemisphere, crs_code

# Example coordinates (for the given image center)
coords = [-84.0162247,34.2023400 ]  # (longitude, latitude)
latitude = coords[1]
longitude = coords[0]

# Get UTM zone and CRS code
utm_zone, hemisphere, crs_code = get_utm_zone_and_crs(latitude, longitude)

print(f"UTM Zone: {utm_zone}, Hemisphere: {hemisphere}, CRS Code: {crs_code}")


UTM Zone: 16, Hemisphere: Northern, CRS Code: EPSG:32616


In [106]:
region

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -84.07053260218788,
              34.247255764205974
            ],
            [
              -84.07053260218788,
              34.157424235794025
            ],
            [
              -83.96191679781214,
              34.157424235794025
            ],
            [
              -83.96191679781214,
              34.247255764205974
            ]
          ]
        ]
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
})

In [112]:

image = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
              .filterBounds(region)
              .filterDate('2020-04-01', '2020-09-01')
            ).first()

In [113]:
proj = image.select('B1').projection().getInfo()
if 'crs' in proj:
    proj_crs = proj['crs']
elif 'wkt' in proj:
    proj_crs = proj['wkt']
p = ee.Projection(proj_crs, proj['transform'])
scale = p.nominalScale().getInfo()

In [114]:
proj_crs

'EPSG:32616'

In [82]:
band = image.getInfo()['bands'][0]
crs = band['crs']
crs_transform = band['crs_transform']
transform = band['crs_transform']
odim = band['dimensions']
origin = 0,0
print(crs)
print(crs_transform)

KeyError: 'dimensions'

In [100]:
proj_crs = crs_code
wp = Proj(proj_crs)
bbox = np.asarray(wp((limit[1], limit[3], limit[3], limit[1]),
                (limit[0], limit[0], limit[2], limit[2]), ))
## move to nearest pixel - 60 m works for both S2 and Landsat
bbox -= bbox % 60
## ROI x and y ranges
xrange = np.asarray((min(bbox[0]), max(bbox[0])))
yrange = np.asarray((max(bbox[1]), min(bbox[1])))
xdim = (xrange[1]-xrange[0])/transform[0]
ydim = (yrange[1]-yrange[0])/transform[4]
## ROI x and y pixel ranges
xprange = (xrange-transform[2]) / transform[0]
yprange = (yrange-transform[5]) / transform[4]
rdim = int(xprange[1]-xprange[0]), int(yprange[1]-yprange[0])
## origin of subset in image
porigin = xprange[0], yprange[0]

In [101]:
porigin,rdim

((769740.0, 3793740.0), (10320, -10260))

In [83]:
band

{'id': 'B1',
 'data_type': {'type': 'PixelType',
  'precision': 'double',
  'min': 0,
  'max': 65535},
 'crs': 'EPSG:4326',
 'crs_transform': [1, 0, 0, 0, 1, 0]}

In [61]:
from pyproj import Proj

nx = origin[0] + odim[0]/2
ny = origin[1] + odim[1]/2
wp = Proj(proj_crs)
mlon, mlat = wp(crs_transform[2]+nx*crs_transform[0], crs_transform[5]+ny*crs_transform[4], inverse=True)
ll = {'longitude': mlon, 'latitude': mlat}
print('Scene centre: {:.5f}E {:.5f}N'.format(ll['longitude'], ll['latitude']))


Scene centre: 58.63019E -21.95210N


In [70]:

def distance_in_ll(lat=0, earth_radius=6378.1370):
    from numpy import pi, cos
    circle_lon=2.*pi*earth_radius # length of one meridian
    onedeglat=(circle_lon/360.) # km in one degree of latitude

    proj_earth_radius=earth_radius*cos(lat*pi/180.)
    circle_lat=2.*pi*proj_earth_radius # length of circle of mean longitude
    onedeglon=(circle_lat/360.) # km in one degree of longitude

    return(onedeglon, onedeglat)
def region_box(site, site_lon, site_lat, box_size = 3 ):
    

    ## get approximate distance per degree lon/lat
    dlon, dlat = distance_in_ll(site_lat)

    ## make box around station
    lat_off = (box_size/dlat)/2
    lon_off = (box_size/dlon)/2
    limit = [site_lat-lat_off, site_lon-lon_off, site_lat+lat_off, site_lon+lon_off]

    return(limit)
    

In [75]:
limit = region_box(None,-84.0162247,34.2023400,10 )

In [84]:
import numpy as np
bbox = np.asarray(wp((limit[1], limit[3], limit[3], limit[1]),
                (limit[0], limit[0], limit[2], limit[2]), ))
## move to nearest pixel - 60 m works for both S2 and Landsat
bbox -= bbox % 60
## ROI x and y ranges
xrange = np.asarray((min(bbox[0]), max(bbox[0])))
yrange = np.asarray((max(bbox[1]), min(bbox[1])))
xdim = (xrange[1]-xrange[0])/transform[0]
ydim = (yrange[1]-yrange[0])/transform[4]
## ROI x and y pixel ranges
xprange = (xrange-transform[2]) / transform[0]
yprange = (yrange-transform[5]) / transform[4]
rdim = int(xprange[1]-xprange[0]), int(yprange[1]-yprange[0])
## origin of subset in image
porigin = xprange[0], yprange[0]

In [86]:
proj_crs

'EPSG:4326'

In [116]:
porigin = (6978.0, 630.0)
rdim = (1032, 1026)
print(porigin)
print(rdim)
tile_size = [606,606]
ptiles = []
tn = 0
tiles_grid = int(np.ceil(rdim[0]/tile_size[0])), int(np.ceil(rdim[1]/tile_size[1]))
num_tiles = tiles_grid[0]*tiles_grid[1]
for ti in range(tiles_grid[0]):
    for tj in range(tiles_grid[1]):
        ti0 = max(porigin[0], porigin[0]+ti*tile_size[0])
        ti1 = min(porigin[0]+(ti+1)*tile_size[0], porigin[0]+rdim[0])#, odim[0])
        tj0 = max(porigin[1], porigin[1]+tj*tile_size[1])
        tj1 = min(porigin[1]+(tj+1)*tile_size[1], porigin[1]+rdim[1])#, odim[1])
        ptiles.append([ti0, ti1, tj0, tj1, 't{}'.format(str(tn).zfill(3))])
        print('tile', ptiles[-1])
        tn+=1
print(transform)
print('Running download with {} tiles'.format(len(ptiles)))

(6978.0, 630.0)
(1032, 1026)
tile [6978.0, 7584.0, 630.0, 1236.0, 't000']
tile [6978.0, 7584.0, 1236.0, 1656.0, 't001']
tile [7584.0, 8010.0, 630.0, 1236.0, 't002']
tile [7584.0, 8010.0, 1236.0, 1656.0, 't003']
[1, 0, 0, 0, 1, 0]
Running download with 4 tiles


In [58]:
collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate('2021-03-01', '2021-03-10') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV')

# Get the first image from the filtered collection
image = collection.first()
print(image.getInfo())
# Define the AOI as a Geometry (in this case, a rectangle)
aoi = ee.Geometry.Rectangle([-122.45, 37.74, -122.4, 37.8])

{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [13012, 13380], 'crs': 'EPSG:32740', 'crs_transform': [40, 0, 408102.2618335261, 0, -40, 7839178.949965716]}], 'version': 1730067777623916, 'id': 'COPERNICUS/S1_GRD/S1A_EW_GRDM_1SDV_20210302T143615_20210302T143719_036820_045451_DE82', 'properties': {'SNAP_Graph_Processing_Framework_GPF_vers': '8.0.1', 'SLC_Processing_facility_org': 'ESA', 'SLC_Processing_facility_country': 'France', 'GRD_Post_Processing_facility_org': 'ESA', 'transmitterReceiverPolarisation': ['VV', 'VH'], 'GRD_Post_Processing_start': 1614703801314, 'sliceNumber': 1, 'GRD_Post_Processing_facility_name': 'Copernicus S1 Core Ground Segment - TLS', 'resolution': 'M', 'SLC_Processing_facility_name': 'Copernicus S1 Core Ground Segment - TLS', 'system:footprint': {'type': 'LinearRing', 'coordinates': [[56.19040776930016, -20.432394407161418], [56.16993280892454, -20.43711702413122], [56.17519413940305, -20.459976

In [13]:
# Region of interest.
coords = [
    -121.58626826832939,
    38.059141484827485,
]
region = ee.Geometry.Point(coords)

# Sentinel-2 median composite.
image = (ee.ImageCollection('COPERNICUS/S2')
              .filterBounds(region)
              .filterDate('2020-04-01', '2020-09-01')
              .median())

# Make a projection to discover the scale in degrees.
proj = ee.Projection('EPSG:4326').atScale(10).getInfo()

# Get scales out of the transform.
scale_x = proj['transform'][0]
scale_y = -proj['transform'][4]

# Make a request object.
request = {
    'expression': image,
    'fileFormat': 'PNG',
    'bandIds': ['B4', 'B3', 'B2'],
    'grid': {
        'dimensions': {
            'width': 640,
            'height': 640
        },
        'affineTransform': {
            'scaleX': scale_x,
            'shearX': 0,
            'translateX': coords[0],
            'shearY': 0,
            'scaleY': scale_y,
            'translateY': coords[1]
        },
        'crsCode': proj['crs'],
    },
    'visualizationOptions': {'ranges': [{'min': 0, 'max': 3000}]},
}

image_png = ee.data.computePixels(request)

In [22]:
from PIL import Image
import io

# Assuming `image_png` is the byte data returned from computePixels
image_bytes = image_png  # Replace with your actual byte data

# Use BytesIO to handle the byte data as an image
image = Image.open(io.BytesIO(image_bytes))

# Save the image as a PNG file
image.save('output_image.png')


In [10]:
# Define the Image, Region, and Bands
i_sr = ee.Image('MODIS/006/MOD13A1/2021_03_01').select('NDVI')
grid = ee.Geometry.Rectangle([-122.45, 37.74, -122.4, 37.8])
obands_sr = ['NDVI']  # Specify the band IDs you want


In [12]:
request = {
    'expression': i_sr.serialize(),       # Serialize the image expression
    'fileFormat': 'GEO_TIFF',             # Output format
    'grid': json.dumps(grid.getInfo()),   # AOI as JSON string
    'bandIds': obands_sr                  # List of bands
}

# Execute computePixels
pixels = ee.data.computePixels(request)

EEException: Invalid value at 'grid' (type.googleapis.com/google.earthengine.v1.PixelGrid), "{"type": "Polygon", "coordinates": [[[-122.45, 37.74], [-122.4, 37.74], [-122.4, 37.8], [-122.45, 37.8], [-122.45, 37.74]]]}"

In [9]:
# Import JSON library for data conversion
import json

# Set up parameters for computePixels
request = {'expression': (i_sr),'fileFormat': 'GEO_TIFF',
            'grid': grid,'bandIds': obands_sr}

# Call computePixels
pixels = ee.data.computePixels(params)

# Print the raw pixel data
print(pixels)


TypeError: Object of type method is not JSON serializable

In [88]:
import math

def calculate_tile_size(n_bands, bytes_per_pixel_type=2, max_size_mb=48):
    # Convert max size to bytes
    max_size_bytes = max_size_mb * 1024 * 1024
    
    # Calculate bytes per pixel with the number of bands
    bytes_per_pixel = bytes_per_pixel_type * n_bands
    
    # Calculate maximum pixels per tile based on the size limit
    max_pixels_per_tile = max_size_bytes / bytes_per_pixel
    
    # Calculate the tile side length (assuming a square tile)
    tile_size = int(math.sqrt(max_pixels_per_tile))
    
    return tile_size
print( calculate_tile_size(1,1,48))

7094


In [103]:
import os,json
from gee_s1_lee import refined_lee
def getGeeData(workspace,extent_gcs,s1_date,tile_size = 606,url_jrc="JRC/GSW1_4/GlobalSurfaceWater",jrc_band = "occurrence",
               s1_band = "VV", url_s1="COPERNICUS/S1_GRD",download_s1=True,download_jrc=True):
    '''
    Download SAR and JRC water images to tiles (default 606 pixels)

    '''
    import ee
    ee.Initialize()
        
    ##Obtain the bouding box by user
    numbers = extent_gcs.split(',')
    s1_date = s1_date.split(',')

    # Assign the extracted numbers to variables
    x1, y1, x2, y2 = map(float, numbers)    
    limit = [y2,x1,y1,x2]
    bbox = ee.Geometry.BBox(x1,y1,x2,y2)
    #Download JRC
    files = []
    try:
        ## Get the image project info
        ## Reproject the image to UTM
        
        longitude, latitude = bbox.centroid(maxError=1).coordinates().getInfo()

        # Get UTM zone and CRS code
        utm_zone, hemisphere, crs_code = get_utm_zone_and_crs(latitude, longitude)

        print(f"UTM Zone: {utm_zone}, Hemisphere: {hemisphere}, CRS Code: {crs_code}")
        utm = ee.Projection(crs_code).atScale(30)
        datasetJRC = ee.Image(url_jrc).select(jrc_band).reproject(utm)
      
        ## Compute how many tiles will be need to download
        from pyproj import Proj
        import numpy as np
        wp = Proj(crs_code)
        user_box = np.asarray(wp((x1,x2,x2,x1),
                        (y1,y1, y2, y2), ))
        user_box -= user_box % 60
        
        west = min(user_box[0])
        east = max(user_box[0])
        north = max(user_box[1])
        south = max(user_box[1])
        bbox = ee.Geometry.Rectangle([west, south, east, north])
        datasetJRC = datasetJRC.clip(bbox)
        proj = datasetJRC.projection().getInfo()
        print(proj)
        if 'crs' in proj:
            proj_crs = proj['crs']
        elif 'wkt' in proj:
            proj_crs = proj['wkt']
        p = ee.Projection(proj_crs, proj['transform'])
        scale = p.nominalScale().getInfo()
        print(f"scale:{scale}")
        transform = proj['transform']

        print(transform)        
        ## ROI x and y ranges
        xrange = np.asarray((min(user_box[0]), max(user_box[0])))
        yrange = np.asarray((min(user_box[1]), max(user_box[1])))
        xdim = (xrange[1]-xrange[0])/transform[0]
        ydim = (yrange[1]-yrange[0])/transform[4]
        ## ROI x and y pixel ranges
        xprange = (xrange-transform[2]) / transform[0]
        yprange = (yrange-transform[5]) / transform[4]
        rdim = int(xprange[1]-xprange[0]), int(yprange[1]-yprange[0])
        ## origin of subset in image
        porigin = xprange[0], yprange[0]
        print(porigin,rdim)
        tile_size = [tile_size,tile_size]
        ptiles = []
        tn = 0
        tiles_grid = int(np.ceil(rdim[0]/tile_size[0])), int(np.ceil(rdim[1]/tile_size[1]))
        num_tiles = tiles_grid[0]*tiles_grid[1]
        for ti in range(tiles_grid[0]):
            for tj in range(tiles_grid[1]):
                ti0 = max(porigin[0], porigin[0]+ti*tile_size[0])
                ti1 = min(porigin[0]+(ti+1)*tile_size[0], porigin[0]+rdim[0])#, odim[0])
                tj0 = max(porigin[1], porigin[1]+tj*tile_size[1])
                tj1 = min(porigin[1]+(tj+1)*tile_size[1], porigin[1]+rdim[1])#, odim[1])
                ptiles.append([ti0, ti1, tj0, tj1, 't{}'.format(str(tn).zfill(3))])
                print('tile', ptiles[-1])
                tn+=1
        print(transform)
        print('Running download with {} tiles'.format(len(ptiles)))
        for ti, tile in enumerate(ptiles):
            ## output file names
            ext = ''
            #if len(rname) >= 0: ext = '_{}'.format(rname)

            print(tile)
            tdim = tile[1]-tile[0], tile[3]-tile[2]
            print(tdim)
            tilex = transform[2] + tile[0] * transform[0]
            tiley = transform[5] + tile[2] * transform[4]
            #print(tile)
            print(tilex, tiley)
            grid = {
                'dimensions': {
                    'width': tdim[0],
                    'height': tdim[1]
                },
                'affineTransform': {
                    'scaleX': transform[0],
                    'shearX': 0,
                    'translateX': tilex,
                    'shearY': 0,
                    'scaleY': transform[4],
                    'translateY': tiley,
                },
                'crsCode': proj_crs,
            }

            print(grid)
            i_sr = datasetJRC
            request = {'expression': (i_sr),
                    'fileFormat': 'GEO_TIFF',
                    'grid': grid,
                    }

            tmp = ee.data.computePixels(request)
            print('st', len(tmp))
            ## write data
            ext = ".tif"
            st_file_tile = "jrc"+f'_st_{ti}'+ext
            file_path = os.path.join(r'C:\tmp',st_file_tile)
            with open(file_path, 'wb') as f:
                f.write(tmp) 
            files.append(file_path)
        #clean up and mosaic
        import rasterio
        from rasterio.merge import merge

        sars = [rasterio.open(image_path) for image_path in files]
        mosaic_array, out_transform = merge(sars)
        profs = [src.profile for src in sars]
        profile = profs[0]

        sar_path = os.path.join(workspace,"jrc.tif")
        with rasterio.open(sar_path, 'w', **profile) as dst:
            dst.write(mosaic_array)

    except Exception as e:
        print(e)
    

       

In [105]:

 
workspace = r'G:\workspace\tmp'
extent = '-80.3248099, 34.9065437,-77.0897903, 36.6986762'

getGeeData(workspace=workspace,extent_gcs=extent,s1_date='2016-10-10,2016-10-30',tile_size=5000)

UTM Zone: 17, Hemisphere: Northern, CRS Code: EPSG:32617
{'type': 'Projection', 'crs': 'EPSG:32617', 'transform': [30, 0, 0, 0, 30, 0]}
scale:30
[30, 0, 0, 0, 30, 0]
(18676.0, 128762.0) (9900, 6856)
tile [18676.0, 23676.0, 128762.0, 133762.0, 't000']
tile [18676.0, 23676.0, 133762.0, 135618.0, 't001']
tile [23676.0, 28576.0, 128762.0, 133762.0, 't002']
tile [23676.0, 28576.0, 133762.0, 135618.0, 't003']
[30, 0, 0, 0, 30, 0]
Running download with 4 tiles
[18676.0, 23676.0, 128762.0, 133762.0, 't000']
(5000.0, 5000.0)
560280.0 3862860.0
{'dimensions': {'width': 5000.0, 'height': 5000.0}, 'affineTransform': {'scaleX': 30, 'shearX': 0, 'translateX': 560280.0, 'shearY': 0, 'scaleY': 30, 'translateY': 3862860.0}, 'crsCode': 'EPSG:32617'}
st 772380
[18676.0, 23676.0, 133762.0, 135618.0, 't001']
(5000.0, 1856.0)
560280.0 4012860.0
{'dimensions': {'width': 5000.0, 'height': 1856.0}, 'affineTransform': {'scaleX': 30, 'shearX': 0, 'translateX': 560280.0, 'shearY': 0, 'scaleY': 30, 'translateY': 4

In [2]:
from osgeo import gdal 
import os
import glob

# Specify the directory containing your GeoTIFF files
data_dir = 'c:/tmp'

tiff_pattern = os.path.join(data_dir, "*.tif")  # Pattern for .tif files
tiff_pattern_tiff = os.path.join(data_dir, "*.tiff")  # Pattern for .tiff files

# List all TIFF files using glob
tiff_files = glob.glob(tiff_pattern) + glob.glob(tiff_pattern_tiff)
first_ds = gdal.Open(tiff_files[0])
# Get the dimensions and geotransform of the first dataset
width = first_ds.RasterXSize
height = first_ds.RasterYSize
geo_transform = first_ds.GetGeoTransform()
projection = first_ds.GetProjection()

# Create the output dataset with the same dimensions
driver = gdal.GetDriverByName('GTiff')
mosaic_ds = driver.Create('output_mosaic.tif', width, height, first_ds.RasterCount, gdal.GDT_Float32)  # Change data type as needed
mosaic_ds.SetGeoTransform(geo_transform)
mosaic_ds.SetProjection(projection)

# Initialize an array to accumulate the mosaic data
mosaic_data = None

# Iterate through each input TIFF file
for tiff in tiff_files:
    # Open each dataset
    ds = gdal.Open(tiff)

    # Read the data
    for band in range(ds.RasterCount):
        data = ds.GetRasterBand(band + 1).ReadAsArray()

        # Create mosaic_data array if it doesn't exist
        if mosaic_data is None:
            mosaic_data = data
        else:
            # Add the new data to the mosaic_data (you can use numpy for more complex operations)
            mosaic_data += data

        # Write the accumulated data to the output band
        mosaic_ds.GetRasterBand(band + 1).WriteArray(mosaic_data)

    # Close the current dataset
    ds = None

# Close the output dataset
mosaic_ds = None

print("Mosaic created successfully: output_mosaic.tif")

ImportError: DLL load failed while importing _gdal_array: The specified procedure could not be found.

In [2]:
tiff_files

[WindowsPath('c:/tmp/jrc_st_0.tif'),
 WindowsPath('c:/tmp/jrc_st_1.tif'),
 WindowsPath('c:/tmp/jrc_st_2.tif'),
 WindowsPath('c:/tmp/jrc_st_3.tif')]

In [33]:
def get_utm_zone_and_crs(lat, lon):
    # Calculate UTM zone number
    zone_number = int((lon + 180) / 6) + 1
    
    # Determine hemisphere and corresponding CRS code
    if lat >= 0:
        hemisphere = "Northern"
        crs_code = f"EPSG:326{zone_number:02d}"  # Northern Hemisphere
    else:
        hemisphere = "Southern"
        crs_code = f"EPSG:327{zone_number:02d}"  # Southern Hemisphere
    
    return zone_number, hemisphere, crs_code